# Entrenamiento del modelo YOLOv8n para validación de EPP

Este notebook implementa el entrenamiento del modelo **YOLOv8n**, una versión consolidada y ampliamente utilizada por la comunidad, reconocida por su **robustez, facilidad de entrenamiento y compatibilidad con TensorRT**.

El propósito es generar un **modelo base de comparación** frente a YOLOv11n y PP-YOLOE, bajo el mismo conjunto de datos de EPP (casco, chaleco, gafas, guantes, mascarilla, zapatos).

El experimento se orienta a obtener un modelo de referencia rápido, con inferencia en tiempo real, que sirva como punto de control en la comparación de rendimiento y precisión.

**Objetivos específicos:**
- Evaluar desempeño del backbone CSPDarknet frente a GELAN.
- Comparar eficiencia energética y latencia con YOLOv11n.
- Exportar pesos entrenados a TensorRT FP16 para Jetson Orin.

In [11]:
from IPython import display

display.clear_output()

import ultralytics

ultralytics.checks()

import torch

print(torch.cuda.is_available())
print(torch.__version__)
print(torch.cuda.get_device_name(0))

Ultralytics 8.3.197 🚀 Python-3.10.16 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32101MiB)
Setup complete ✅ (32 CPUs, 62.5 GB RAM, 85.1/457.3 GB disk)
True
2.8.0+cu128
NVIDIA GeForce RTX 5090


In [10]:
!nvidia-smi

Wed Oct 29 11:38:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:01:00.0  On |                  N/A |
| 31%   47C    P8             44W /  600W |     775MiB /  32607MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
from ultralytics import YOLO

In [1]:
# Entrenamiento
!yolo detect train model=/home/ose/Documents/ppe-detection/src/yolov8n.pt data=/home/ose/Documents/ppe-detection/ppe.yaml epochs=100 batch=64 plots=True device=0 name=PPE_yolo8n patience=30 warmup_epochs=3 save_period=10

New https://pypi.org/project/ultralytics/8.3.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.197 🚀 Python-3.10.16 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32101MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/ose/Documents/ppe-detection/ppe.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/home/ose/Documents/ppe-detection/src/yolov8n.pt, momentum=0.937, mosaic=

In [ ]:
# Exportación a TensorRT
!yolo export model=runs/detect/PPE_yolo8n/weights/best.pt format=engine device=0 half=True